# CIFAR dataset 

The intention of this noteboook is purely for fun! i wanted to experiment with trying different hyperparameters on a dataset i am not familar with

From Wikipedia, the free encyclopedia: """

The CIFAR-10 dataset (Canadian Institute For Advanced Research) is a collection of images that are commonly used to train machine learning and computer vision algorithms. It is one of the most widely used datasets for machine learning research.[1][2] The CIFAR-10 dataset contains 60,000 32x32 color images in 10 different classes.[3] The 10 different classes represent airplanes, cars, birds, cats, deer, dogs, frogs, horses, ships, and trucks. There are 6,000 images of each class.[4]

Computer algorithms for recognizing objects in photos often learn by example. CIFAR-10 is a set of images that can be used to teach a computer how to recognize objects. Since the images in CIFAR-10 are low-resolution (32x32), this dataset can allow researchers to quickly try different algorithms to see what works. Various kinds of convolutional neural networks tend to be the best at recognizing the images in CIFAR-10.

CIFAR-10 is a labeled subset of the 80 million tiny images dataset. When the dataset was created, students were paid to label all of the images.[5]

"""

In [1]:
import os 
import sklearn
import numpy as np 
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [2]:
path = "/content/drive/MyDrive/Colab Notebooks"
os.chdir(path)
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks'

####CIFAR10 dataset.

You can load it with keras.datasets.cifar10.load_data(). 
The dataset is composed of 60,000 32 × 32–pixel color images (50,000 for training, 10,000 for testing) with 10 classes, so you'll need a softmax output layer with 10 neurons. 
Remember to search for the right learning rate each time you change the model's architecture or hyperparameters.

In [3]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]
keras.backend.clear_session()

In [4]:
### building a very large model for learning purposes. then we will mess with it to improve results 
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 activation="elu",
                                 kernel_initializer="he_normal"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 100)               307300    
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 dense_3 (Dense)             (None, 100)               10100     
                                                                 
 dense_4 (Dense)             (None, 100)               10100     
                                                                 
 dense_5 (Dense)             (None, 100)               1

##### Callbacks & scheduler

In [13]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1**(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(lr0=0.001, s=25)
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [7]:
optimizer = keras.optimizers.Nadam(learning_rate=5e-5)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [8]:
tf.random.set_seed(42)
np.random.seed(42)
model.fit(X_train, y_train, epochs=1000,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

Epoch 1/1000
1407/1407 [==============================] - 18s 10ms/step - loss: 6.0695 - accuracy: 0.1663 - val_loss: 2.2285 - val_accuracy: 0.2034
Epoch 2/1000
1407/1407 [==============================] - 14s 10ms/step - loss: 2.0745 - accuracy: 0.2404 - val_loss: 2.2745 - val_accuracy: 0.2178
Epoch 3/1000
1407/1407 [==============================] - 14s 10ms/step - loss: 1.9605 - accuracy: 0.2808 - val_loss: 2.0664 - val_accuracy: 0.2572
Epoch 4/1000
1407/1407 [==============================] - 14s 10ms/step - loss: 1.8877 - accuracy: 0.3087 - val_loss: 1.9183 - val_accuracy: 0.2938
Epoch 5/1000
1407/1407 [==============================] - 14s 10ms/step - loss: 1.8274 - accuracy: 0.3324 - val_loss: 1.8302 - val_accuracy: 0.3382
Epoch 6/1000
1407/1407 [==============================] - 14s 10ms/step - loss: 1.7732 - accuracy: 0.3574 - val_loss: 1.8098 - val_accuracy: 0.3502
Epoch 7/1000
1407/1407 [==============================] - 14s 10ms/step - loss: 1.7187 - accuracy: 0.3765 - val_

In [9]:
## this allows a look at the eval of the best model 
model = keras.models.load_model("my_cifar10_model.h5")
model.evaluate(X_valid, y_valid)

157/157 [==============================] - 1s 2ms/step - loss: 1.4753 - accuracy: 0.4822


[1.4752825498580933, 0.4821999967098236]

In [16]:
#adding learning_rate callback 
keras.backend.clear_session()

callbacks2 = [early_stopping_cb,lr_scheduler]

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 activation="elu",
                                 kernel_initializer="he_normal"))
model.add(keras.layers.Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

model.fit(X_train, y_train, epochs=1000,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks2)


Epoch 1/1000
1407/1407 [==============================] - 16s 9ms/step - loss: 9.1680 - accuracy: 0.2054 - val_loss: 2.0166 - val_accuracy: 0.2422 - lr: 0.0010
Epoch 2/1000
1407/1407 [==============================] - 12s 9ms/step - loss: 1.9620 - accuracy: 0.2638 - val_loss: 2.0107 - val_accuracy: 0.2542 - lr: 9.1201e-04
Epoch 3/1000
1407/1407 [==============================] - 12s 9ms/step - loss: 1.9112 - accuracy: 0.2948 - val_loss: 1.9431 - val_accuracy: 0.2802 - lr: 8.3176e-04
Epoch 4/1000
1407/1407 [==============================] - 12s 9ms/step - loss: 1.8815 - accuracy: 0.3082 - val_loss: 2.0640 - val_accuracy: 0.2548 - lr: 7.5858e-04
Epoch 5/1000
1407/1407 [==============================] - 12s 9ms/step - loss: 1.8533 - accuracy: 0.3193 - val_loss: 1.8190 - val_accuracy: 0.3352 - lr: 6.9183e-04
Epoch 6/1000
1407/1407 [==============================] - 12s 9ms/step - loss: 1.8274 - accuracy: 0.3286 - val_loss: 1.7895 - val_accuracy: 0.3352 - lr: 6.3096e-04
Epoch 7/1000
1407/14